# Gene Set and Pathway

![](./images/Module3/Workflow3.jpg)

Differential expression (DE) analysis typically yields a list of genes or proteins. Our intention is to use such lists to gain novel insights about genes and proteins that may have roles in a given
phenomenon, phenotype or disease progression. However, in many cases, gene lists generated from DE analysis are
difficult to interpret due to their large size and lack of useful annotations. Hence, pathway analysis (also known
as gene set analysis or over-representation analysis), aims to reduce the complexity of interpreting gene lists
via mapping the listed genes to known (i.e. annotated) biological pathways, processes and functions.
This learning submodule introduces common curated biological databases including Gene Ontology (GO) and the
Kyoto Encyclopedia of Genes and Genomes (KEGG).

## Learning Objectives:
1. Introduction to Ontology and Gene Ontology.
2. Introduction to KEGG Pathway Database.
3. Download terms, pathway gene set from GO and KEGG.
4. Save results to GMT file format.

In [1]:
IRdisplay::display_html('<iframe src="./Quizzes/Quiz_Submodule3-1.html" width=100% height=250></iframe>')

## Ontology and Gene Ontology
### Overview
In this section, we will learn about the concept of gene ontology in bioinformatics. Ontology is a set of concepts and categories defined by a shared vocabulary to denote  properties of the concepts, as well as the relationships between the concepts.
Ontology plays an important role in the field of bioinformatics. Ontology enables unambiguous communication e.g.,
a way to understand different groups’ annotations of various genomes. Also, it allows the knowledge to be structured to perform automated analyses by computer programs.

The Gene Ontology (GO) database defines a structured, common, and controlled vocabulary to describe attributes of genes and gene products
across organisms. Collaboration is key to build a consensus vocabulary. But the term gene ontology, or GO, is commonly used
to refer to both the terms as well as the associations between genes, which is sometimes a source of potential confusion. In order to avoid this, here we will use the term “GO”
to describe the set of terms and their hierarchical structure and “GO annotations” to describe the set of associations between
genes and GO terms. The GO is divided into three categories to describe the genes and gene products from three different
angles: Molecular Function, Biological Process, and Cellular Component.

The structure of GO can be described in terms of directed acyclic graphs (DAGs), where each GO term is a node,
and the relationships between the terms are edges. GO is loosely hierarchical, with ‘child’ terms
being more specialized than their ‘parent’ terms, but unlike a strict hierarchy, a term may have more than one parent
term (note that the parent/child model does not hold for all types of relations). The structure of the controlled
vocabularies is intended to reflect true, biological relationships. In contrast to strict hierarchies, DAGS allows
multiple relationships between a more granular (child) term and a more general parent term. The relationship between
terms affects how queries are made. For example, a query for all genes with binding activity would include transcription
factors as well as genes with other types of binding activity (such as protein binding, ligand binding). The illustration
of the category and structure of GO is shown in the figure below:

![](./images/Module3/GO_Structure.jpg)
*(Source: https://www.ebi.ac.uk/, http://geneontology.org/)*

### Gene ontology relationship
In DAGs graph, *terms* are represented as *nodes* and *relations* (also known as *object properties*) between the *terms*
are *edges*. There are commonly used relationships in GO such as *is a* (is a *subtype of*), *part of, has part, regulates,
negatively regulates and positively regulates*. All terms (except for the root terms representing each aspect) have a sub-class relationship to another term.

Examples:

<div class="alert alert-block alert-warning">
    <i class="fa fa-pencil" aria-hidden="true"></i>
    <b>Example:</b>  
    
**GO:1904659:glucose transport** *is* a **GO:0015749:monosaccharide transport**.

The *is a* relation forms the basic structure of GO. If we say A *is a* B, we mean that node A is a subtype of node B

</div>

<div class="alert alert-block alert-warning">
    <i class="fa fa-pencil" aria-hidden="true"></i>
    <b>Example:</b> 
    
**GO:0031966:mitochondrial membrane** *is part of* **GO:0005740:mitochondrial envelope**

The *part of* relation is used to represent part-whole relationships. A *part of* relation would only be added between
A and B if B is **necessarily** *part of* A: wherever B exists, it is as *part of* A, and the presence of the B implies
the presence of A. However, given the occurrence of A, we cannot say for certain that B exists.
<div>

<div class="alert alert-block alert-warning">
    <i class="fa fa-pencil" aria-hidden="true"></i>
    <b>Example:</b> 
    
**GO:0098689:latency-replication decision** *regulates* **GO:0019046:release from viral latency**

A relation that describes a case in which one process directly affects the manifestation of another process or quality,
i.e. the former *regulates* the latter.
    <div>


A more specific case with more nodes and edges can be seen in the figure below:
<br>
![](./images/Module3/GO_Relation.jpg)
*(Source: https://advaitabio.com/)* <br>
For more technical information about relations and their properties used in GO and other ontologies, see the
<a href="https://obofoundry.org/ontology/ro.html">OBO Relations Ontology (RO)</a>


In [2]:
#Run the following command to take the quiz
IRdisplay::display_html('<iframe src="Quizzes/Quiz_Submodule3.html" width=100% height=250></iframe>')

### GO storage file formats
GO terms are updated monthly in the following formats:
* OBO 1.4 files are human-readable (in addition to machine-readable) and can be opened in any text editor.
* OWL files can be read by <a href="https://protege.stanford.edu/">Protégé</a> text editor.

 In this learning submodule, we will only use ".OBO" to obtain GO terms. The OBO file format is for representing ontologies and controlled vocabularies. The format itself attempts to achieve the following goals:
 * Human readability
 * Ease of parsing
 * Extensibility
 * Minimal redundancy

The file structure is shown in the following figure.


![](./images/Module3/OBO_Format.jpg)

The OBO file has a header, which is an unlabeled section at the beginning of the document. The header ends when the first term is encountered. Next, terms are represented in a labeled section with the tag *[Term]*. Under each term, we can find other information such as term ID, official name, category (namespace), term definition, synonym and relation to other GO terms.

At this step, we still don't know what genes are related to which GO terms. In order to retrieve custom sets of gene ontology annotations for any list of genes from organisms, NCBI has published a Gene2GO database that obtains GO terms and the entrez gene IDs related to those GO terms. The database can be retrieved from <a href="https://ftp.ncbi.nih.gov/gene/DATA/gene2go.gz">here</a>. The Gene2GO database can be viewed using a text editor, the file structure is presented in the figure below:

![](./images/Module3/Gene2GO.jpg)

The OBO and Gene2GO databases will be used in combination to obtain GO terms and related genes for enrichment analysis.

### Retrieving GO terms from the DE gene list
This section focuses on downloading related GO terms based on the DE gene list obtained from the DE analysis in the previous section.
Here, we will use `topGO` and `hgu133plus2.db` R packages to obtain GO terms. The `topGO` package has built-in functions that use Gene2GO databases to retrieve GO terms from the gene ID given by DE analysis. Since the dataset we used in submodule 02 was generated for humans, we will use `hgu133plus2.db` database to map probeIDs to gene symbols.
The installation process of two packages can be done by the script below:

In [1]:
# Installation of topGO and hgu133plus2.db package
suppressMessages({if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
  suppressWarnings(BiocManager::install("topGO", update = F))
  suppressWarnings(BiocManager::install("hgu133plus2.db", update = F))
  suppressWarnings(BiocManager::install("AnnotationDbi", update = F))
})

In [2]:
# Loading the library
suppressPackageStartupMessages({
  suppressWarnings(library("topGO"))
  suppressWarnings(library("hgu133plus2.db"))
  suppressWarnings(library("AnnotationDbi"))
})


groupGOTerms: 	GOBPTerm, GOMFTerm, GOCCTerm environments built.



Load the DE genelist generated from the DE analysis using `limma`.

In [3]:
# Loading the DE result
data = readRDS("./data/DE_genes.rds")

By default, the DE analysis performed by `limma` contains multiple features. However, adjusted *p-value* and *gene ID* are the most important features for enrichment analysis. We can use the following code to list all of the gene IDs and their *p-value*.

In [4]:
# Get p-value from DE results
genelist <- data$adj.P.Val
# Assign gene IDs to associated p-values
names(genelist) <- data$PROBEID

After successfully obtaining the gene list, we need to map the gene IDs to the gene symbols using `hgu133plus2.db`.

In [9]:
# Map gene IDs to gene symbols
gene <- suppressMessages(AnnotationDbi::select(hgu133plus2.db, keys = names(genelist), columns = "SYMBOL"))

In [7]:
# Remove duplicated gene IDs
gene <- gene[!duplicated(gene[, 1]),]

# Assign the result to a new gene list with gene symbols
geneList2 <- genelist
names(geneList2) <- gene[, 2]

Now, we can search for related GO terms based on the new gene list using `topGO` package. First, we need to create a `topGOdata`
object.

In [10]:
# Retrieve all the GO terms related to the gene list obtained from the expression matrix
GOdata <- new("topGOdata", description = "", ontology = "BP",
                    allGenes = geneList2, geneSel = function(x)x, nodeSize = 10,
                    annot = annFUN.org, ID = "alias", mapping = "org.Hs.eg")


Building most specific GOs .....

	( 12272 GO terms found. )


Build GO DAG topology ..........

	( 15583 GO terms and 35092 relations. )


Annotating nodes ...............

	( 16314 genes annotated to the GO terms. )



We can search for related GO terms using `geneInTerm` function and view the term with associated genes.

In [11]:
# Obtain a list of genes for each GO term
allGO = genesInTerm(GOdata)
allGO[1:5]

$`GO:0000002`
 [1] "AKT3"     "DNA2"     "DNAJA3"   "ENDOG"    "FLCN"     "LIG3"    
 [7] "LONP1"    "MDP1"     "MEF2A"    "METTL4"   "MGME1"    "MPV17"   
[13] "OPA1"     "PARP1"    "PIF1"     "PIM1"     "POLB"     "POLG"    
[19] "POLG2"    "PPARGC1A" "PRIMPOL"  "RRM1"     "RRM2B"    "SESN2"   
[25] "SLC25A33" "SLC25A36" "SLC25A4"  "SSBP1"    "STOX1"    "TOP3A"   
[31] "TP53"     "TWNK"     "TYMP"    

$`GO:0000003`
   [1] "A1CF"         "A2M"          "AAAS"         "ABAT"        
   [5] "ABCC8"        "ABHD2"        "ACE"          "ACE2"        
   [9] "ACOD1"        "ACOX1"        "ACR"          "ACRBP"       
  [13] "ACRV1"        "ACSBG2"       "ACSL4"        "ACTL7A"      
  [17] "ACTL9"        "ACTR2"        "ACTR3"        "ACVR1"       
  [21] "ACVR1B"       "ACVR1C"       "ACVR2A"       "ADA"         
  [25] "ADAD1"        "ADAD2"        "ADAM15"       "ADAM18"      
  [29] "ADAM2"        "ADAM20"       "ADAM21"       "ADAM28"      
  [33] "ADAM29"       "ADAM30"       "ADAM32"       "ADAMTS1"     
  [37] "ADAMTS16"     "ADAMTS2"      "ADCY10"       "ADCY3"       
  [41] "ADCY7"        "ADCYAP1"      "ADCYAP1R1"    "ADGB"        
  [45] "ADGRG1"       "ADGRG2"       "ADIG"         "ADM"         
  [49] "ADNP"         "ADRA2A"       "ADRA2B"       "AFF4"        
  [53] "AFG2A"        "AFP"          "AGFG1"        "AGO2"        
  [57] "AGO4"         "AGRP"         "AKAP3"        "AKAP4"       
  [61] "AKR1C3"       "AKT1"         "ALDOA"        "ALKBH5"      
  [65] "ALOX15B"      "ALPL"         "AMBP"         "AMD1"        
  [69] "AMH"          "AMHR2"        "ANAPC1"       "ANAPC10"     
  [73] "ANAPC11"      "ANAPC13"      "ANAPC15"      "ANAPC16"     
  [77] "ANAPC2"       "ANAPC4"       "ANAPC5"       "ANAPC7"      
  [81] "ANG"          "ANGPT2"       "ANKLE1"       "ANKRD31"     
  [85] "ANKRD49"      "ANTXR1"       "ANXA1"        "AP3B1"       
  [89] "APC2"         "APELA"        "APLF"         "APOB"        
  [93] "APOL2"        "APOL3"        "APP"          "AR"          
  [97] "AREG"         "ARHGDIB"      "ARID4A"       "ARID4B"      
 [101] "ARID5B"       "ARMC12"       "ARMC2"        "ASB1"        
 [105] "ASF1B"        "ASH1L"        "ASIP"         "ASPM"        
 [109] "ASZ1"         "ATAT1"        "ATM"          "ATN1"        
 [113] "ATP1A1"       "ATP1A4"       "ATP2B2"       "ATP2B4"      
 [117] "ATP7A"        "ATP8B3"       "ATR"          "ATRX"        
 [121] "AURKA"        "AURKC"        "AVP"          "AVPR1A"      
 [125] "AXDND1"       "AXIN1"        "AXL"          "AZI2"        
 [129] "AZIN2"        "B4GALNT1"     "B4GALT1"      "BACH1"       
 [133] "BAG6"         "BAK1"         "BAP1"         "BASP1"       
 [137] "BAX"          "BBS1"         "BBS2"         "BBS4"        
 [141] "BCAP31"       "BCL2"         "BCL2L1"       "BCL2L10"     
 [145] "BCL2L11"      "BCL2L2"       "BCL6"         "BIK"         
 [149] "BIRC3"        "BMAL1"        "BMP15"        "BMP4"        
 [153] "BMP5"         "BMP6"         "BMP7"         "BMPR1A"      
 [157] "BMPR1B"       "BMPR2"        "BNC1"         "BOK"         
 [161] "BOLL"         "BPY2"         "BRCA2"        "BRD2"        
 [165] "BRDT"         "BRINP1"       "BRIP1"        "BRME1"       
 [169] "BSG"          "BTBD18"       "BUB3"         "C14orf39"    
 [173] "C16orf92"     "C1QBP"        "C2CD6"        "C3"          
 [177] "C3orf62"      "C9orf78"      "CA12"         "CABS1"       
 [181] "CABYR"        "CACNA1H"      "CAD"          "CADM1"       
 [185] "CALCA"        "CALR"         "CALR3"        "CAPN2"       
 [189] "CASP2"        "CASP3"        "CASP8"        "CATSPER1"    
 [193] "CATSPER2"     "CATSPER3"     "CATSPERB"     "CATSPERD"    
 [197] "CATSPERE"     "CATSPERG"     "CBL"          "CBS"         
 [201] "CBX2"         "CCDC136"      "CCDC182"      "CCDC34"      
 [205] "CCDC38"       "CCDC39"       "CCDC40"       "CCDC42"      
 [209] "CCDC62"       "CCDC63"       "CCDC87"       "CCIN"        
 [213] "CCNA1"        "CCNB

Now, we already had GO terms with genes. However, we still do not know the meaning of GO terms related to biological processes. We can use `GO.db` database to get a set of annotation maps describing the entire Gene Ontology assembled using data from GO. We can use the following code to install the `GO.db` R package.

In [12]:
suppressMessages({if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
  suppressWarnings(BiocManager::install("GO.db", update = F))
})

In [13]:
library(GO.db)

Then, we can use the following commands to obtain the GO terms description.

In [13]:
# Getting the name of each GO term
terms <- names(allGO)
# Getting the description of each GO term
descriptions <-lapply(Term(terms), `[[`, 1)

In order to perform enrichment analysis in later submodules, we need to save the GO terms and genesets to the standard output. One commonly used format is Gene Matrix Transposed file format *(\*.gmt)*. The GMT file format is a tab delimited file format that describes gene sets. In the GMT format, each row represents a gene set; in the GMX format, each column represents a gene set. Here, we can save GO terms and genesets to the *\*.gmt* using the following function:

In [14]:
# A function to save the GO terms with genesets to the local storage
writeGMT <- function(genesets, descriptions, outfile) {

  if (file.exists(outfile)) {
    file.remove(outfile)
  }
  for (gs in names(genesets)) {
    write(c(gs, gsub("\t", " ", descriptions[[gs]]), genesets[[gs]]), file=outfile, sep="\t", append=TRUE, ncolumns=length(genesets[[gs]]) + 2)
  }
}
outfile <- "./data/GO_terms.gmt"
writeGMT(allGO, descriptions, outfile)

In [ ]:
# save the GO terms with genesets to Google Cloud Bucket
# replace <BUCKET_NAME> with name of your bucket that was previously made in submodule 1
system("gsutil cp ./data/GO_terms.gmt gs://<BUCKET_NAME>", intern = TRUE)

## Kyoto Encyclopedia of Genes and Genomes (KEGG)
### Overview
KEGG is a collection of databases dealing with genomes, biological pathways, diseases, drugs, and chemical substances. KEGG is utilized for bioinformatics research and education, including data analysis in genomics, metagenomics, metabolomics and other omics studies, modeling and simulation in systems biology, and translational research in drug development. The KEGG database project was initiated in 1995 by Minoru Kanehisa, professor at the Institute for Chemical Research, Kyoto University, under the then ongoing Japanese Human Genome Program. Foreseeing the need for a computerized resource that can be used for biological interpretation of genome sequence data, he started developing the KEGG PATHWAY database. It is a collection of manually drawn KEGG pathway maps representing experimental knowledge on metabolism and various other functions of the cell and the organism. Each pathway map contains a network of molecular interactions and reactions and is designed to link genes in the genome to gene products (mostly proteins) in the pathway. This has enabled the analysis called KEGG pathway mapping, whereby the gene content in the genome is compared with the KEGG PATHWAY database to examine which pathways and associated functions are likely to be encoded in the genome. KEGG is a "computer representation" of the biological system. It integrates building blocks and wiring diagrams of the system—more specifically, genetic building blocks of genes and proteins, chemical building blocks of small molecules and reactions, and wiring diagrams of molecular interaction and reaction networks. The illustrative structure of KEGG is presented as figure below.
![](./images/Module3/KEGG.jpg)


In [15]:
#Run the following command to take the quiz
IRdisplay::display_html('<iframe src="./Quizzes/Quiz_Submodule3-2.html" width=100% height=250></iframe>')

### Retrieving pathways from KEGG databases
In this section, we will retrieve pathways and related genesets from the KEGG database using R commands. Here we will use the `KEGGREST` R package that provides a client interface to the KEGG REST server. `KEGGREST` can be installed from the Bioconductor using the following command.

In [16]:
suppressMessages({if (!require("BiocManager", quietly = TRUE))
    install.packages("BiocManager")
  suppressWarnings(BiocManager::install("KEGGREST", update = F))
})

In [17]:
suppressPackageStartupMessages({
  library(KEGGREST)
})

KEGG contains a number of databases. To get an idea of what is available, run `listDatabases()`:

In [18]:
KEGGREST::listDatabases()

[1] "pathway"  "brite"    "module"   "ko"       "genome"   "vg"      
 [7] "ag"       "compound" "glycan"   "reaction" "rclass"   "enzyme"  
[13] "disease"  "drug"     "dgroup"   "environ"  "genes"    "ligand"  
[19] "kegg"

We can use these databases in further queries. Note that in many cases you can also use a three-letter KEGG organism code or a “T number” (genome identifier) in the same place you would use one of these database names.

We can obtain the list of organisms available in KEGG with the `keggList()` function:

In [19]:
organism <- keggList("organism")

In [20]:
print(paste0("KEGG supports ", dim(organism)[1], " organisms"))

[1] "KEGG supports 9576 organisms"


To view the supported organisms we can use the following command:

In [21]:
# View several supported organism
head(organism)

T.number,organism,species,phylogeny
T01001,hsa,Homo sapiens (human),Eukaryotes;Animals;Vertebrates;Mammals
T01005,ptr,Pan troglodytes (chimpanzee),Eukaryotes;Animals;Vertebrates;Mammals
T02283,pps,Pan paniscus (bonobo),Eukaryotes;Animals;Vertebrates;Mammals
T02442,ggo,Gorilla gorilla gorilla (western lowland gorilla),Eukaryotes;Animals;Vertebrates;Mammals
T01416,pon,Pongo abelii (Sumatran orangutan),Eukaryotes;Animals;Vertebrates;Mammals
T03265,nle,Nomascus leucogenys (northern white-cheeked gibbon),Eukaryotes;Animals;Vertebrates;Mammals


In [22]:
IRdisplay::display_html('<iframe src="./Quizzes/Quiz_Submodule3-3.html" width=100% height=250></iframe>')

In submodule 02, we performed DE analysis on a human dataset. Therefore, we need to download pathways for humans. The abbreviation of human pathway in KEGG is `hsa` and we can use the `keggList` function to get the pathway list.

In [23]:
# Obtain the pathways belongs to human
pathways.list <- keggList("pathway", "hsa")

The pathway list contains pathway description and pathway code in a single line of text. To see the first five pathways related to **carbohydrate metabolism** we will use the commands below.

In [24]:
#list the specific pathways to view
pathway_ids <- c("hsa00010", "hsa00020", "hsa00030", "hsa00040", "hsa00051")
#view their names
pathways.list[pathway_ids]

hsa00010 
            "Glycolysis / Gluconeogenesis - Homo sapiens (human)" 
                                                         hsa00020 
               "Citrate cycle (TCA cycle) - Homo sapiens (human)" 
                                                         hsa00030 
               "Pentose phosphate pathway - Homo sapiens (human)" 
                                                         hsa00040 
"Pentose and glucuronate interconversions - Homo sapiens (human)" 
                                                         hsa00051 
         "Fructose and mannose metabolism - Homo sapiens (human)"

To learn more about these pathways you can visit https://www.genome.jp/kegg/pathway.html. 

In [25]:
#Run the following command to take the quiz
IRdisplay::display_html('<iframe src="./Quizzes/Quiz_Submodule3-4.html" width=100% height=250></iframe>')

We can use the following command to check how many pathways are available for human.

In [26]:
# Retrieve all the pathway IDs belong to human
pathway.codes <- sub("path:", "", names(pathways.list))
pathway.codes

[1] "hsa01100" "hsa01200" "hsa01210" "hsa01212" "hsa01230" "hsa01232"
  [7] "hsa01250" "hsa01240" "hsa00010" "hsa00020" "hsa00030" "hsa00040"
 [13] "hsa00051" "hsa00052" "hsa00053" "hsa00500" "hsa00520" "hsa00620"
 [19] "hsa00630" "hsa00640" "hsa00650" "hsa00562" "hsa00190" "hsa00910"
 [25] "hsa00920" "hsa00061" "hsa00062" "hsa00071" "hsa00100" "hsa00120"
 [31] "hsa00140" "hsa00561" "hsa00564" "hsa00565" "hsa00600" "hsa00590"
 [37] "hsa00591" "hsa00592" "hsa01040" "hsa00230" "hsa00240" "hsa00250"
 [43] "hsa00260" "hsa00270" "hsa00280" "hsa00290" "hsa00310" "hsa00220"
 [49] "hsa00330" "hsa00340" "hsa00350" "hsa00360" "hsa00380" "hsa00400"
 [55] "hsa00410" "hsa00430" "hsa00440" "hsa00450" "hsa00470" "hsa00480"
 [61] "hsa00510" "hsa00513" "hsa00512" "hsa00515" "hsa00514" "hsa00532"
 [67] "hsa00534" "hsa00533" "hsa00531" "hsa00563" "hsa00601" "hsa00603"
 [73] "hsa00604" "hsa00511" "hsa00730" "hsa00740" "hsa00750" "hsa00760"
 [79] "hsa00770" "hsa00780" "hsa00785" "hsa00790" "hsa00670" "hsa00830"
 [85] "hsa00860" "hsa00130" "hsa00900" "hsa00232" "hsa00524" "hsa00980"
 [91] "hsa00982" "hsa00983" "hsa03020" "hsa03022" "hsa03040" "hsa03010"
 [97] "hsa00970" "hsa03013" "hsa03015" "hsa03008" "hsa03060" "hsa04141"
[103] "hsa04130" "hsa04120" "hsa04122" "hsa03050" "hsa03018" "hsa03030"
[109] "hsa03410" "hsa03420" "hsa03430" "hsa03440" "hsa03450" "hsa03460"
[115] "hsa03082" "hsa03083" "hsa03250" "hsa03260" "hsa03264" "hsa03265"
[121] "hsa03266" "hsa03267" "hsa02010" "hsa04010" "hsa04012" "hsa04014"
[127] "hsa04015" "hsa04310" "hsa04330" "hsa04340" "hsa04350" "hsa04390"
[133] "hsa04392" "hsa04370" "hsa04371" "hsa04630" "hsa04064" "hsa04668"
[139] "hsa04066" "hsa04068" "hsa04020" "hsa04070" "hsa04072" "hsa04071"
[145] "hsa04024" "hsa04022" "hsa04151" "hsa04152" "hsa04150" "hsa04080"
[151] "hsa04060" "hsa04061" "hsa04512" "hsa04514" "hsa04144" "hsa04145"
[157] "hsa04142" "hsa04146" "hsa04140" "hsa04136" "hsa04137" "hsa04148"
[163] "hsa04110" "hsa04114" "hsa04210" "hsa04215" "hsa04216" "hsa04217"
[169] "hsa04115" "hsa04218" "hsa04510" "hsa04520" "hsa04530" "hsa04540"
[175] "hsa04550" "hsa04814" "hsa04810" "hsa04640" "hsa04610" "hsa04611"
[181] "hsa04613" "hsa04620" "hsa04621" "hsa04622" "hsa04623" "hsa04625"
[187] "hsa04650" "hsa04612" "hsa04660" "hsa04658" "hsa04659" "hsa04657"
[193] "hsa04662" "hsa04664" "hsa04666" "hsa04670" "hsa04672" "hsa04062"
[199] "hsa04911" "hsa04910" "hsa04922" "hsa04923" "hsa04920" "hsa03320"
[205] "hsa04929" "hsa04912" "hsa04913" "hsa04915" "hsa04914" "hsa04917"
[211] "hsa04921" "hsa04926" "hsa04935" "hsa04918" "hsa04919" "hsa04928"
[217] "hsa04916" "hsa04924" "hsa04614" "hsa04925" "hsa04927" "hsa04260"
[223] "hsa04261" "hsa04270" "hsa04970" "hsa04971" "hsa04972" "hsa04976"
[229] "hsa04973" "hsa04974" "hsa04975" "hsa04979" "hsa04977" "hsa04978"
[235] "hsa04962" "hsa04960" "hsa04961" "hsa04964" "hsa04966" "hsa04724"
[241] "hsa04727" "hsa04725" "hsa04728" "hsa04726" "hsa04720" "hsa04730"
[247] "hsa04723" "hsa04721" "hsa04722" "hsa04744" "hsa04740" "hsa04742"
[253] "hsa04750" "hsa04360" "hsa04380" "hsa04211" "hsa04213" "hsa04710"
[259] "hsa04713" "hsa04714" "hsa05200" "hsa05202" "hsa05206" "hsa05205"
[265] "hsa05204" "hsa05207" "hsa05208" "hsa05203" "hsa05230" "hsa05231"
[271] "hsa05235" "hsa05210" "hsa05212" "hsa05225" "hsa05226" "hsa05214"
[277] "hsa05216" "hsa05221" "hsa05220" "hsa05217" "hsa05218" "hsa05211"
[283] "hsa05219" "hsa05215" "hsa05213" "hsa05224" "hsa05222" "hsa05223"
[289] "hsa05166" "hsa05170" "hsa05161" "hsa05160" "hsa05171" "hsa05164"
[295] "hsa05162" "hsa05168" "hsa05163" "hsa05167" "hsa05169" "hsa05165"
[301] "hsa05110" "hsa05120" "hsa05130" "hsa05132" "hsa05131" "hsa05135"
[307] "hsa05133" "hsa05134" "hsa05150" "hsa05152" "hsa05100" "hsa05146"
[313] "hsa05144" "hsa05145" "hsa05140" "hsa05142" "hsa05143" "hsa05310"
[319] "hsa05322" "hsa05323" "hsa05320" "hsa05321" "hsa05330" "hsa05332"
[325] "hsa05340" "hsa05010" "hsa05012" "hsa05014" "hsa05016" "hsa05017"
[331] "hsa05020" "hsa05022" "hsa05030" "hs

In [27]:
print(paste0("Number of available pathways for human are: ", length(pathway.codes)))

[1] "Number of available pathways for human are: 356"


We can see that, in each line, the text in the quotation mark contains pathway information while the later part contains pathway code leading by a prefix `path:`. To get pathway codes from the pathway list, we can use the following commands:

Use the following code to get a list of genes and pathway's description for all pathways available in human.

In [29]:
# Function to get all the gene names for each pathway
genes.by.pathway <- sapply(pathway.codes,
                           function(pwid){
                             pw <- keggGet(pwid)
                             if (is.null(pw[[1]]$GENE)) return(NA)
                             pw2 <- pw[[1]]$GENE[c(FALSE,TRUE)]
                             pw2 <- unlist(lapply(strsplit(pw2, split = ";", fixed = T), function(x)x[1]))
                             return(pw2)
                           }
)
# Function to get description for each pathway
description.by.pathway <- sapply(pathway.codes,
                                 function(pwid){
                                   pw <- keggGet(pwid)
                                   if (is.null(pw[[1]]$NAME)) return(NA)
                                   pw2 <- pw[[1]]$NAME
                                   return(pw2)
                                 }
)
# Convert the pathway description to a list
description.by.pathway <- as.list(description.by.pathway)

Something to note is that the KEGG database also provides pathways that hold many sub pathways so when we look up the pathways through keggGet, they may not have any genes listed in that report. This can skew our results so the for loop below takes out any pathway that does not have a Gene section from the list.

We can view the first five pathways with their genesets using the following command

In [30]:
# View the five pathways with the genesets
genes.by.pathway <- genes.by.pathway[!is.na(genes.by.pathway)]
genes.by.pathway[1:5]

$hsa00010
 [1] "HK3"     "HK1"     "HK2"     "HKDC1"   "GCK"     "GPI"     "PFKM"   
 [8] "PFKP"    "PFKL"    "FBP1"    "FBP2"    "ALDOC"   "ALDOA"   "ALDOB"  
[15] "TPI1"    "GAPDH"   "GAPDHS"  "PGK2"    "PGK1"    "PGAM1"   "PGAM2"  
[22] "PGAM4"   "ENO3"    "ENO2"    "ENO1"    "ENO4"    "PKM"     "PKLR"   
[29] "PDHA2"   "PDHA1"   "PDHB"    "DLAT"    "DLD"     "LDHAL6A" "LDHAL6B"
[36] "LDHA"    "LDHB"    "LDHC"    "ADH1A"   "ADH1B"   "ADH1C"   "ADH7"   
[43] "ADH4"    "ADH5"    "ADH6"    "AKR1A1"  "ALDH2"   "ALDH3A2" "ALDH1B1"
[50] "ALDH7A1" "ALDH9A1" "ALDH3B1" "ALDH3B2" "ALDH3A1" "ACSS1"   "ACSS2"  
[57] "GALM"    "PGM1"    "PGM2"    "G6PC1"   "G6PC2"   "G6PC3"   "ADPGK"  
[64] "BPGM"    "MINPP1"  "PCK1"    "PCK2"   

$hsa00020
 [1] "CS"     "ACLY"   "ACO2"   "ACO1"   "IDH1"   "IDH2"   "IDH3B"  "IDH3G" 
 [9] "IDH3A"  "OGDHL"  "OGDH"   "DLST"   "DLD"    "SUCLG1" "SUCLG2" "SUCLA2"
[17] "SDHA"   "SDHB"   "SDHC"   "SDHD"   "FH"     "MDH1"   "MDH2"   "PC"    
[25] "PCK1"   "PCK2"   "PDHA2"  "PDHA1"  "PDHB"   "DLAT"  

$hsa00030
 [1] "GPI"     "G6PD"    "PGLS"    "H6PD"    "PGD"     "RPE"     "RPEL1"  
 [8] "TKT"     "TKTL2"   "TKTL1"   "TALDO1"  "RPIA"    "SHPK"    "DERA"   
[15] "RBKS"    "PGM1"    "PGM2"    "PRPS1L1" "PRPS2"   "PRPS1"   "RGN"    
[22] "IDNK"    "GLYCTK"  "ALDOC"   "ALDOA"   "ALDOB"   "FBP1"    "FBP2"   
[29] "PFKM"    "PFKP"    "PFKL"   

$hsa00040
 [1] "GUSB"    "KL"      "UGT2A1"  "UGT2A3"  "UGT2B17" "UGT2B11" "UGT2B28"
 [8] "UGT1A6"  "UGT1A4"  "UGT1A1"  "UGT1A3"  "UGT2B10" "UGT1A9"  "UGT2B7" 
[15] "UGT1A10" "UGT1A8"  "UGT1A5"  "UGT2B15" "UGT1A7"  "UGT2B4"  "UGT2A2" 
[22] "UGDH"    "UGP2"    "AKR1A1"  "CRYL1"   "RPE"     "RPEL1"   "XYLB"   
[29] "AKR1B1"  "AKR1B10" "AKR1B15" "DCXR"    "SORD"    "DHDH"    "FGGY"   
[36] "CRPPA"  

$hsa00051
 [1] "MPI"     "PMM2"    "PMM1"    "GMPPB"   "GMPPA"   "GMDS"    "GFUS"   
 [8] "FPGT"    "FCSK"    "ENOSF1"  "HK3"     "HK1"     "HK2"     "HKDC1"  
[15] "PFKM"    "PFKP"    "PFKL"    "FBP1"    "FBP2"    "PFKFB1"  "PFKFB2" 
[22] "PFKFB3"  "PFKFB4"  "TIGAR"   "KHK"     "SORD"    "AKR1B1"  "AKR1B10"
[29] "AKR1B15" "ALDOC"   "ALDOA"   "ALDOB"   "TPI1"    "TKFC"

Use the following command to see the description of the first five pathways

In [31]:
# View the description of the first five pathways
description.by.pathway <- description.by.pathway[!is.na(description.by.pathway)]
description.by.pathway[names(genes.by.pathway[1:5])]

$hsa00010
[1] "Glycolysis / Gluconeogenesis - Homo sapiens (human)"

$hsa00020
[1] "Citrate cycle (TCA cycle) - Homo sapiens (human)"

$hsa00030
[1] "Pentose phosphate pathway - Homo sapiens (human)"

$hsa00040
[1] "Pentose and glucuronate interconversions - Homo sapiens (human)"

$hsa00051
[1] "Fructose and mannose metabolism - Homo sapiens (human)"

Taking a look at the file since the first 8 pathways do not have any genes listed we can remove those hsa ids.

In [32]:
# Saving the pathway information to the local repository
outfile <- "./data/KEGG_pathways.gmt"
writeGMT(genes.by.pathway, description.by.pathway, outfile)

In [ ]:
# Saving the pathway information to the Google Cloud Bucket
# replace <BUCKET_NAME> with name of your bucket that was perviously made in submodule 1
system("gsutil cp ./data/KEGG_pathways.gmt gs://<BUCKET_NAME>", intern = TRUE)

In the next submodule, we will do Pathway Analysis.


In [ ]:
sessionInfo()